<a href="https://colab.research.google.com/github/Svetorus/Neuronets/blob/master/Neuronet__%22%D0%9F%D0%B0%D0%BC%D1%8F%D1%82%D1%8C_%D1%81%D0%B2%D0%B5%D1%82%D0%B0%22_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Продолжение книги "Алиса в стране чудес" с помощью рекуррентной нейросети.

In [0]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [0]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.models import Sequential


# построчное чтение из примера с текстом 
with open("Память_света.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("utf-8","ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)

# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 50, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 5 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 5
NUM_PREDS_PER_EPOCH = 100


# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. 
За ним последует полносвязный слой связанный с набором возможных следующих символов, 
которые конвертированы в вероятностные результаты через стандартную softmax активацию с 
multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''

model = Sequential()
model.add(
    LSTM(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()


Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Итерация #: 0

Epoch 1/5
711041/711041 [==============================] - 333s 469us/step - loss: 2.3694
Epoch 2/5
711041/711041 [==============================] - 341s 479us/step - loss: 1.9695
Epoch 3/5
711041/711041 [==============================] - 337s 474us/step - loss: 1.8065
Epoch 4/5
711041/711041 [==============================] - 340s 479us/step - loss: 1.7110
Epoch 5/5
711041/711041 [==============================] - 344s 484us/step - loss: 1.6511
Генерация из посева: роны и, будем надеяться, доберутся до южных ворот.
роны и, будем надеяться, доберутся до южных ворот. — Я прошлось в ного не сторону троллоков на него не стояли в ного на сторону и продолжил свои свои ==================================================
Итерация #: 1
Epoch 1/5
711041/711041 [==============================] - 342s 481us/step - loss: 1.6087
Epoch 2/5
711041/711041 [==============================] - 338s 475us/step - 